In [ ]:
# %config InlineBackend.figure_format = 'retina'
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set()

# Homework 4

Austin Gill

* 5: 2, 10abd
* 9: 1, 2
* 10: 1, 2
* Text addition

## Problem 5.2

Using Veranda and the basic motion algorithm, place a set of obstacles that cause the robot to cycle and not find the goal. In other words, build a robot trap.

## Problem 5.10.a

Implement Bug 1 in Veranda. Screen capture the results.

## Problem 5.10.b

Implement Bug 2 in Veranda. Screen capture the results.

## Problem 5.10.d

Implement Tangent Bug in Veranda. Screen capture the results.

## Problem 9.1

Assume that you are working in a large event center which has beacons located around the facility. Estimate the location of a robot, $(a,b,c)$, if the $(x,y,z) $ location of the beacon and the distance from the beacon to the robot, $d$, are given in the table below.

| $x$ | $y$ | $z$ | $d$ |
|-----|-----|-----|-----|
| 884 | 554 | 713 | 222 |
| 120 | 703 | 771 | 843 |
| 938 | 871 | 583 | 436 |
| 967 | 653 | 46  | 529 |
| 593 | 186 | 989 | 610 |

First, we plot the spheres to get a good idea of what we're looking at.

In [ ]:
centers = [
    (884, 554, 713),
    (120, 703, 771),
    (938, 871, 583),
    (967, 653, 46),
    (593, 186, 989),
]

radii = [222, 843, 436, 529, 610]

spheres = list(zip(centers, radii))

In [ ]:
u = np.linspace(0, 2 * np.pi, 20)
v = np.linspace(0, np.pi, 20)

circles = []

# Generate the points on the spheres
for center, r in spheres:
    x = center[0] + r * np.outer(np.cos(u), np.sin(v))
    y = center[1] + r * np.outer(np.sin(u), np.sin(v))
    z = center[2] + r * np.outer(np.ones(np.size(u)), np.cos(v))
    circles.append((x, y, z))

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')
ax.view_init(azim=-10, elev=15)

for x, y, z in circles:
    ax.plot_surface(x, y, z, linewidth=0.05, alpha=0.6)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

# TODO: <rant></rant>
# ax.set_xlim(0, 1000) and ax.set_xbound(0, 1000) set the *axis* bounds
# not the data bounds, so the spheres still get plotted outside the axis...
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)
ax.set_zlim(0, 1000)

plt.show()

Now how to find the intersection point? For simplicity's sake, I will use the optimization technique given as the first example in the textbook. For this, we build an error function

$$f(x, y, z) = \sum_i \left( \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - r_i \right)^2$$

and find the point $(x, y, z)$ that minimizes the error and claim that that point is as close to an intersection of the spheres as we're going to get.

I will modify this error function to also take in a vector $\vec s$ of sphere indices to use. This way I can loop through all combinations of 3, 4, and 5 of the given spheres.

In [ ]:
def E(v, s):
    """Error function for the given set of spheres."""
    x, y, z = v
    e = 0
    for sphere in (spheres[i] for i in s):
        (xi, yi, zi), ri = sphere
        e += (np.sqrt((x - xi)**2 + (y - yi)**2 + (z - zi)**2) - ri)**2

    return e

In [ ]:
# A generator of all the sphere index vectors
ss = itertools.chain.from_iterable(itertools.combinations(range(5), i) for i in [3, 4, 5])

In [ ]:
results = []
for s in ss:
    results.append(sp.optimize.minimize(E, x0=(800, 400, 400), args=(s,)))

Note that the `minimize` function can fail, in which case the returned `OptimizeResult` object has the `success` field set to `False`.

In [ ]:
# Lists of (x, y, z) tuples
positions = np.array([r.x for r in results if r.success])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Unzip the list of tuples into a tuple of lists
ax.scatter(*zip(*positions), alpha=0.7)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.title('Optimization results for all combinations of 3, 4, and 5 spheres')

plt.show()

This gives us a bunch of potential intersection points. Yay. So we remove the two outliers and plot the centroid.

In [ ]:
x, y, z = zip(*positions)

filtered = []

# Filter out the outliers
for xi, yi, zi in zip(x, y, z):
    if xi > 860:
        filtered.append((xi, yi, zi))

filtered = np.array(filtered)

centroid = np.mean(filtered, axis=0)
print(f'centroid: {centroid}')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Unzip the list of tuples into a tuple of lists
ax.scatter(*zip(*filtered), alpha=0.7, label='Intersections')
ax.scatter(*centroid, label='Centroid')

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.title('Optimization results with outliers removed')
plt.legend()

plt.show()

So the estimated robot position is near $(886.03, 442.21, 521.77)$. If matplotlib would respect the `ax.set_*lim()` function calls when I plot the sphere surfaces, I could generate a helpful plot showing the spheres and the estimated position. But I've spent far too much of my life going through the first ten pages of my Google searches (I even used Google over DuckDuckGo for this) for me to try any more.

## Problem 9.2

If you are using a laser diode to build a distance sensor, you need some method to determine the travel time. Instead of using pulses and a clock, try using phase shifts. What is the wavelength of the modulated frequency of 10MHz? If you measure a 10 degree phase shift, this value corresponds to what distances? What if the phase shift measurement has noise: zero mean with standard deviation 0.1? How does one get a good estimate of position if the ranges to be measured are from 20 meters to 250 meters?

We have wavelength $\lambda = \frac{c}{f}$, where $c = 3 \times 10^8$ and $f = 10\times10^6$, so then $\lambda = 30$m.

A $10^\circ$ phase shift is $\frac{10}{360} = \frac{1}{36}$ of the wavelength, so with a wave length of $30$m, we have an estimate of the distance travelled of $30\cdot\frac{1}{36} = 0.8\overline{33}$ meters for the round trip distance.

Now, of course, realise that the phase shift would be the same for any round trip distance $0.8\overline{33} + 30n$ where $n \in \mathbb Z$.

If the phase shift reading is additively normally noisy, then the above value of $\frac{1}{36}$ will be replaced by some distribution with a mean and standard deviation. So we'd have instead, $X + 30n$, where $X$ is random. However, the size difference of $X$ relative to the size of $30n$ for almost any $n$ is small! So for small amounts of noise, our overall distance estimate (for this particular problem) is not severly impacted.

If we know something about our arena, we can place limits on the $n$ from above. Namely, $n \in \{1, \dots, 8\}$. However, this still gives too much uncertainty. So we should modulate the laser at two *relatively prime* frequencies so phase shifts will only coincide at a single unique distance.

Pick $17$MHz, which gives $\lambda = \frac{c}{f} = 17.6471$. Then $D_2 = \lambda \frac{\theta_2}{360} + m \lambda$. Then we find $n, m \in \mathbb Z$ for which $\lambda \frac{\theta_1}{360} + n \lambda = \lambda \frac{\theta_2}{360} + m \lambda$. Note, however, that this requires a second phase shift measurement $\theta_2$, which may have additional noise.

In a noisy system, we may (will) not have an exact equality between $D_1$ and $D_2$, so we'd have to find values of $n$ and $m$ for which they agree within some tolerance. We'd have to determine the tolerance based on the size of our arena (the scale at which we're working at) and the amount of noise we believe we see.

## Problem 10.1

Assume you have a laser triangulation system as shown in Fig. 10.2 given by (10.1) and that $f=8$mm, $b=30$cm. What are the ranges for $\alpha$ and $u$ if we need to measure target distances in a region (in cm) $20<z<100$ and $10<x<30$?

$$x = \frac{b u}{f\cot \alpha + u},  \quad
  z = \frac{b f}{f\cot \alpha + u}$$
  
![Fig 10.2](figures/lasertriangulation2.svg)

This is an exercise in $\LaTeX$ (a.k.a., algebra).

We have

$$\begin{cases}
x &= \frac{bu}{f \cot \alpha + u} \\
z &= \frac{bf}{f \cot \alpha + u}
\end{cases}$$

Solve this system of equations for $u$ and $\alpha$, then find the minimum and maximum values for $u$ and $\alpha$ as $x$ and $z$ range over the given domain.

Starting with $z$, we have

$$\begin{align*}
z &= \frac{bf}{f \cot \alpha + u} \\
z(f \cot \alpha + u) &= bf \\
zu &= bf - zf \cot \alpha \\
u &= \frac{bf}{z} - f \cot \alpha
\end{align*}$$

Then substitute $u = \frac{bf}{z} - f \cot \alpha$ into

$$\require{cancel}\begin{align*}
x &= \frac{bu}{f \cot \alpha + u} \\
  &= \frac{b\left(\frac{bf}{z} - f \cot \alpha\right)}{f \cot \alpha + \frac{bf}{z} - f \cot \alpha} \\
  &= \frac{\bcancel{f}b \left(\frac{b}{z} - \cot \alpha\right)}{\bcancel{f}\left(\cancel{\cot \alpha} + \frac{b}{z} - \cancel{\cot \alpha}\right)} \\
  &= \frac{b \left(\frac{b}{z} - \cot \alpha\right)}{\left(\frac{b}{z}\right)} \\
  &= \frac{\frac{b^2}{z} - b \cot \alpha}{\frac{b}{z}} \\
  &= \frac{z}{b}\left(\frac{b^2}{z} - b \cot \alpha\right) \\
  &= b - z \cot \alpha \\
z \cot \alpha &= b - x \\
\cot \alpha &= \frac{b - x}{z} \\
\alpha &= \cot^{-1}\left(\frac{b - x}{z}\right)
\end{align*}$$

So barring any algebra mistakes, we have the new (old, with new look) system of equations

$$\begin{cases}
u &= \frac{fx}{z} \\
\alpha &= \cot^{-1}\left(\frac{b - x}{z}\right)
\end{cases}$$

Now we can range over $10 < x < 30$ and $20 < z < 100$ to find the minimum and maximum values of $u$ and $\alpha$.

Since $b = 30$cm and $f = 0.8$cm, our system of equations only has two unknowns. Further, each component is relatively simple, so the minimum and maximum values of $u$ and $\alpha$ can be immediately picked out.

Since $x$ and $z$ are both positive, $u$ will have its minimum value at $(x, z) = (10, 100)$ of $\frac{0.8 \cdot 10}{100} = 0.08$ and its maximum value at $(x, z) = (30, 20)$ of $\frac{0.8 \cdot 30}{20} = 1.2$.

Similarly, since $\cot^{-1} x$ is strictly positive and monotonically decreasing over $x \in[0, \infty)$, $\alpha$ is greatest as the smallest value of $\frac{b - x}{z}$ and least at the largest value of $\frac{b - x}{z}$. So the largest value of $\alpha$ is $\cot^{-1}(0) = \frac{\pi}{2}$, and the smallest value of $\alpha$ is $\cot^{-1}\left(\frac{30 - 10}{20}\right) = \cot^{1} = \frac{\pi}{4}$.

So in summary, we found

$$\begin{cases}
u &= \frac{fx}{z} \\
\alpha &= \cot^{-1}\left(\frac{b - x}{z}\right)
\end{cases}$$

where $0.08 < u < 1.2$ and $\frac{\pi}{4} < \alpha < \frac{\pi}{2}$.

## Problem 10.2

Assume you have two cameras that are calibrated into a stereo pair with a baseline of $10$cm, and focal depth of $7$mm. If the error is $10\%$ on $v_1$ and $v_2$, $v_1=2$mm and $v_2=3$mm, what is the error on the depth measurement $z$? Your answer should be a percentage relative to the error free number. Hint: If $v_1=2$ then a $10\%$ error ranges from $1.8$ to $2.2$. (Although not required, another way to approach this problem is the total differential from calculus.)

Similar to problem 10.1, let $1.8 < v_1 < 2.2$ and $2.7 < v_2 < 3.3$ and determine the range of values $x$ and $z$ take.

$$\begin{cases}
x &= \frac{v_1 b}{v_1 + v_2} \\
z &= \frac{fb}{v_1 + v_2}
\end{cases}$$

Because the denominator of each of these fractions contains *both* $v_1$ and $v_2$, we can expect some curvature in the plotted surfaces. This means that we cannot just plug in the boundary of the domain and expect to find the extrema. However, if we look at the plot of $x(v_1, v_2)$ and $z(v_1, v_2)$ it appears we get lucky and can easily pick out the extrema. Wolfram Alpha agrees.

In [ ]:
# Change units from mm to cm.
v1 = np.linspace(0.18, 0.22, 10)
v2 = np.linspace(0.27, 0.33, 10)

v1, v2 = np.meshgrid(v1, v2)

x = v1 * 10 / (v1 + v2)
z = 0.7 * 10 / (v1 + v2)

In [ ]:
print(f'min x = {x.min()}, max x = {x.max()}')
print(f'min z = {z.min()}, max z = {z.max()}')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(azim=45, elev=30)

plt.title('$x$ as a function of $v_1$ and $v_2$')
ax.plot_surface(v1, v2, x, linewidth=0, alpha=0.8)
# ax.plot_surface(v1, v2, x.min() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')
# ax.plot_surface(v1, v2, x.max() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')

ax.set_xlabel('$v_1$')
ax.set_ylabel('$v_2$')
ax.set_zlabel('$x$')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
# ax.view_init(azim=45, elev=30)

plt.title('$z$ as a function of $v_1$ and $v_2$')
ax.plot_surface(v1, v2, z, linewidth=0, alpha=0.8)
# ax.plot_surface(v1, v2, z.min() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')
# ax.plot_surface(v1, v2, z.max() + np.zeros_like(v1), linewidth=0, alpha=0.3, color='k')

ax.set_xlabel('$v_1$')
ax.set_ylabel('$v_2$')
ax.set_zlabel('$z$')

plt.show()

In [ ]:
print(12.727 / 14)
print(15.556 / 14)

The minimum of $z(v_1, v_2)$ can be found at $(v_1, v_2) = (0.22, 0.33)$ with value $z = 12.\overline{72}$. The maximum occurs at $(v_1, v_2) = (0.18, 0.27)$ with value $z = 15.\overline{55}$.

So $z$ has range $12.727 < z < 15.556$ with $10\%$ error on both $v_1$ and $v_2$. The actual value is $z = 14$cm.

Since percentages are hard, I always have to do

$$14 p_l = 12.727$$

and

$$14 p_h = 15.556$$

to get proportions $p_l = 0.909$ and $p_h = 1.111$. In the region close to the actual value of $z = 14$cm, we can assume linearity, and claim that the error in the measured $z$ value versus the actual value is $\sim 10\%$. But note that the error is will grow as $v_1$ and $v_2$ decrease due to the curvature of the surface, which becomes dramatic for larger ranges of $v_1$ and $v_2$.